In [1]:
# notebook to compute log squared error of a mechanism compared to all data points from RCM experiments

In [2]:
import os
import sys
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures
import rmgpy.chemkin
import subprocess

In [3]:
# load the mechanism
mech = 1
base_rmg = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/chem_annotated.cti'
improved_rmg = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/cutoff3_20230505_top50.cti'
aramco = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/aramco/AramcoMech3.0.cti'

In [4]:

# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def run_simulation(T, P, X):
    # function to run a RCM simulation

    # gas is a global object
    t_end = 1.0  # time in seconds
    base_gas.TPX = T, P, X

    reactor = ct.IdealGasReactor(base_gas)
    reactor_net = ct.ReactorNet([reactor])

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        try:
            reactor_net.step()
        except ct._cantera.CanteraError:
            print('Reactor failed to solve!')
            print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            return 0

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    delay_i = np.argmax(slopes)
    return times[delay_i]

In [5]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
# ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)
table_exp = df_exp[df_exp['Table'] < 13]
# Define Initial conditions using experimental data
tau_exp = table_exp['time (ms)'].values.astype(float)  # ignition delay
T7 = table_exp['T_C'].values  # Temperatures
P7 = table_exp['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
phi7 = table_exp['phi'].values  # equivalence ratios
# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
for i in range(0, len(phi7)):
    if phi7[i] == 0.3:
        x_diluent = 0.7821
        conc_dict = {
            'O2(2)': 0.2083,
            'butane(1)': 0.00962
        }
    elif phi7[i] == 0.5:
        x_diluent = 0.7771
        conc_dict = {
            'O2(2)': 0.2070,
            'butane(1)': 0.01595
        }
    elif phi7[i] == 1.0:
        x_diluent = 0.7649
        conc_dict = {
            'O2(2)': 0.2038,
            'butane(1)': 0.03135
        }
    elif phi7[i] == 2.0:
        x_diluent = 0.7416
        conc_dict = {
            'O2(2)': 0.1976,
            'butane(1)': 0.06079
        }
    else:
        raise ValueError
    if mech == 3:
        o2_conc = conc_dict.pop('O2(2)')
        conc_dict['O2'] = o2_conc

        butane_conc = conc_dict.pop('butane(1)')
        conc_dict['C4H10'] = butane_conc
    
    x_N2 = table_exp['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table_exp['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table_exp['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    if mech < 3:
        conc_dict['CO2(7)'] = x_CO2
    else:
        conc_dict['CO2'] = x_CO2
    concentrations.append(conc_dict)
assert len(T7) == len(concentrations)

In [6]:
T7

array([ 746,  845,  876,  901,  920,  978, 1042, 1048, 1074, 1077,  683,
        707,  707,  750,  750,  801,  800,  854,  852,  886,  885,  913,
        911,  925,  926,  984,  985, 1053, 1052,  685,  685,  707,  707,
        754,  754,  801,  801,  858,  858,  882,  883,  933,  935,  993,
        994,  710,  732,  732,  889,  931,  929,  929,  973,  969, 1030,
       1031,  663,  664,  713,  713,  737,  737,  782,  782,  814,  847,
        848,  884,  885,  937,  939,  980,  980,  666,  665,  666,  715,
        714,  738,  737,  783,  782,  849,  854,  892,  951,  950,  700,
        701,  731,  732,  756,  757,  794,  795,  840,  849,  872,  881,
        906,  926,  929,  959,  703,  702,  734,  736,  761,  761,  804,
        804,  850,  849,  886,  886,  935,  936,  701,  701,  701,  676,
        676,  696,  696,  729,  729,  767,  764,  796,  796,  822,  823,
        679,  678,  701,  702,  733,  733,  769,  769,  679,  677,  677,
        690])

In [7]:
table_exp['Table'].values

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
        9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11, 11, 11, 11, 12, 12, 12, 12])

In [8]:
# # compute and save the delays
# print('Running Base RMG Delays')
# base_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(base_rmg)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         base_rmg_delays[condition_index] = delay_time
# np.save('base_rmg_delays.npy', base_rmg_delays)

In [9]:
# print('Running Improved RMG Delays')
# improved_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(improved_rmg)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         improved_rmg_delays[condition_index] = delay_time
# np.save('improved_rmg_delays.npy', improved_rmg_delays)

In [10]:
# print('Running Aramco Delays')
# aramco_delays = np.zeros(len(concentrations))
# improved_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(aramco)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         aramco_delays[condition_index] = delay_time
# np.save('aramco_delays.npy', aramco_delays)

In [11]:
base_rmg_delays = np.load('base_rmg_delays_parallel.npy')
improved_rmg_delays = np.load('improved_rmg_delays_parallel.npy')
aramco_delays = np.load('aramco_delays_parallel.npy')

In [12]:
assert np.sum(base_rmg_delays == 0) == 0
assert np.sum(improved_rmg_delays == 0) == 0
assert np.sum(aramco_delays == 0) == 0

In [13]:
tau_exp[tau_exp == 0] = np.nan

In [14]:
def calc_log_squared_error(mech_delays):
    return np.nansum(np.float_power(np.log(mech_delays) - np.log(tau_exp / 1000.0), 2.0))

In [15]:
calc_log_squared_error(base_rmg_delays)

56.17963991877858

In [16]:
calc_log_squared_error(improved_rmg_delays)

44.03354015546501

In [17]:
calc_log_squared_error(aramco_delays)

46.4540249465436

In [29]:
def calc_total_avg_log_squared_error():
    exp_delays = tau_exp[table_exp['Table'].values < 13]
    
    mech_delays = base_rmg_delays[table_exp['Table'].values < 13]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Base RMG: {np.round(avg_err, 3)}')
    print(f'{np.round(avg_err, 5)}')
    
    mech_delays = improved_rmg_delays[table_exp['Table'].values < 13]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Improved RMG: {np.round(avg_err, 3)}')
    print(f'{np.round(avg_err, 5)}')
    
    mech_delays = aramco_delays[table_exp['Table'].values < 13]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Aramco: {np.round(avg_err, 4)}')
    print(f'{np.round(avg_err, 5)}')
#     print(table_exp['Table'].values == table_index)
    

In [31]:
calc_total_avg_log_squared_error()

0.07678
0.06018
0.06349


In [32]:
(0.07678 - 0.06018) / 0.07678

0.21620213597290966

In [27]:
def calc_avg_log_squared_error(table_index):
    exp_delays = tau_exp[table_exp['Table'].values == table_index]
    
    mech_delays = base_rmg_delays[table_exp['Table'].values == table_index]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Base RMG: {np.round(avg_err, 3)}')
    print(f'{np.round(avg_err, 5)}')
    
    mech_delays = improved_rmg_delays[table_exp['Table'].values == table_index]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Improved RMG: {np.round(avg_err, 3)}')
    print(f'{np.round(avg_err, 5)}')
    
    mech_delays = aramco_delays[table_exp['Table'].values == table_index]
    avg_err = np.nanmean(np.float_power(np.log10(mech_delays) - np.log10(exp_delays / 1000.0), 2.0))
#     print(f'Aramco: {np.round(avg_err, 4)}')
    print(f'{np.round(avg_err, 5)}')
#     print(table_exp['Table'].values == table_index)
    

In [28]:
for i in range(1, 13):
    print(i)
    calc_avg_log_squared_error(i)
    print()

1
0.06456
0.06484
0.10383

2
0.04328
0.03697
0.02372

3
0.05387
0.04241
0.04351

4
0.17044
0.17003
0.28272

5
0.06596
0.05031
0.0515

6
0.10226
0.07927
0.06512

7
0.0555
0.0345
0.03537

8
0.04901
0.03346
0.00943

9
0.1501
0.11127
0.14127

10
0.12574
0.0776
0.04005

11
0.03442
0.02328
0.0362

12
0.09901
0.08243
0.0826



In [ ]:
tau_exp

In [ ]:
base_rmg_delays

In [ ]:
table_exp[table_exp['Table'] == 1]['time (ms)']

In [ ]:
aramco_mech = ct.Solution(aramco)

In [ ]:
aramco_mech.species_names.index('C4H10')